In [ ]:
pip install kafka-python

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [ ]:
#1-Creating producer & sending data to consumer 
try:
    producer = KafkaProducer(
        bootstrap_servers=['54.91.216.170:9092'],
        value_serializer=lambda x: dumps(x).encode('utf-8')
    )
    print("Producer created successfully")
except Exception as e:
    print("Producer error:", e)

In [ ]:
#check EC2 Public IP Connected to KAFKA- if True then only it will work
print(producer.bootstrap_connected())

In [ ]:
#1-Creating producer & sending data to consumer 
def on_success(record):
    print("Message delivered:", record)

def on_error(excp):
    print("ERROR:", excp)

producer.send('demo_testing_6', 
value={'hi':'kiran'}).add_callback(on_success).add_errback(on_error)

producer.flush()

In [ ]:
#2-read data from csv 
df = pd.read_csv("C:\\Users\\DELL\\Documents\\DataPipeline_Projects\\Stock_Market_6\\indexProcessed.csv")

In [ ]:
df.head()

In [ ]:
#2-read data from csv & upload to s3
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_testing_6', value=dict_stock)
    sleep(1)

In [ ]:
#clear data from kafka server
producer.flush()